In [1]:
import sys
import os
# sys.path.append(top) 把392頂層目錄加到最前面 '/home/pan/django_projects/fast_api_392'
top = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.insert(0, top)
# sys.path

In [2]:
from apps.book.classes.bbooks import BOOKS

{'store': None, 'bookid': None, 'isbn10': None, 'isbn13': None, 'title': None, 'title2': None, 'author': None, 'publisher': None, 'pub_dt': None, 'lang': None, 'price_list': None, 'price_sale': None, 'spec': None, 'intro': None, 'comment': None, 'url_book': None, 'url_vdo': None, 'url_cover': None, 'err': None, 'create_dt': None}
{'bookid': '0010770978'}


In [ ]:
init = {'bookid': '0010550825'}
book = BOOKS(**init)

In [ ]:
# # book.update_errcnt = 0
# await book.update_info()
# # book.__dict__
dict.fromkeys(['a', 'b'])

In [ ]:
import pandas as pd
df = pd.DataFrame({'a': [1, 2], 'b': [3, 4]})
df

In [ ]:
a = {1: None}

In [ ]:
a[1] = 1

In [ ]:
a

In [ ]:
# rtext2
a = '123{}'
a.format(456)

In [ ]:
class INFO:

    def __setattr__(self, name, value):
        self.__dict__[name] = value

In [ ]:
a = INFO()

In [ ]:
a.e = 1

In [ ]:
a.__dict__

In [ ]:
b.test

In [ ]:
object.__setattr__(b, 'test', 101)

In [ ]:
object.__getattribute__(b, 'test')

In [ ]:
class A:
    a = 1

In [ ]:
class B(A):
    b = 22
    pass

In [ ]:
B.__dict__